In [1]:
import pandas as pd
from collections import defaultdict
from pathlib import Path
from bs4 import BeautifulSoup
import requests
import csv 
import re
from langdetect import detect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
import json
import nltk
import re
import heapq
import math
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Luca
[nltk_data]     Palluzzi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
f = open("homework.txt", "w")

for i in range(1,301):
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=" + str(i))
    soup = BeautifulSoup(page.content, features="lxml")
    links = soup.find_all('a', itemprop='url', class_='bookTitle')
    for link in links:
        fullLink = link.get('href')
        f.write('https://www.goodreads.com' + fullLink + '\n')
f.close()

In [13]:
path = 'D:/Storage file PC/Documenti/Università/Data Science/Anno 1/Semestre 1/Algorthmic Methods of Data Mining/Homeworks/HW3/st/'

f = open("homework.txt", "r")

ff = f.readlines()

for i in range(30000, len(ff)+1):
    
    folderName = "folder-" + str(i) + "/"
    fileName = "article_" + str(i) + ".html"
    

    url = ff[i-1]
    
    Path(path + folderName).mkdir(parents=True, exist_ok=True)

    page = requests.get(url)
    code = str(page.text)

    with open(path + folderName + fileName, "w", encoding="utf-8") as z:
        z.write(code)

    z.close()
f.close()

In [15]:
personalPath = "D:/Storage file PC/Documenti/Università/Data Science/Anno 1/Semestre 1/Algorthmic Methods of Data Mining/Homeworks/HW3/st/folder-"

In [ ]:
data = ['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', "ratingCount",\
        "reviewCount", "plot", "numberOfPages", "publishingDate", "characters", "setting", "url"]

personalPath = "D:/Storage file PC/Documenti/Università/Data Science/Anno 1/Semestre 1/Algorthmic Methods of Data Mining/Homeworks/HW3/st/folder-"

for i in range(18001, 30001):
    with open(personalPath + str(i) + "/article_" + str(i) + ".html", 'rb') as html: 
        soup = BeautifulSoup(html,"html.parser")
    #plot = soup.find('div',id='description').text.strip()  
    try:
        plot = soup.find('div',id='description').text.strip()
        if detect(plot)=='en':
            lista=[]

            #title
            try:
                lista.append(soup.find('h1').text.strip())
            except:
                lista.append('')

                #bookseries
            try:
                lista.append(soup.find('h2',id='bookSeries').text.strip())
            except:
                lista.append('')

                #author name
            try:
                lista.append(soup.find('a',class_='authorName').text.strip())
            except:
                lista.append('')

                #rating value
            try:
                lista.append(soup.find('span', itemprop='ratingValue').text.strip())
            except:
                lista.append('')

                #ratingCount
            try:
                lista.append(soup.find_all('a',class_='gr-hyperlink',href='#other_reviews')[0]\
                             .text.strip().replace('\r', '').replace('\n', '').split()[0])
            except:
                lista.append('')

                #reviewCount
            try:
                lista.append(soup.find_all('a',class_='gr-hyperlink',href='#other_reviews')[1]\
                             .text.strip().replace('\r', '').replace('\n', '').split()[0])
            except:
                lista.append('')

                #plot
            try:
                if plot[-7:] == '...more':
                    lista.append(soup.find('div',id='description').contents[3].text)
                else:
                    lista.append(plot)
            except:
                lista.append('')

                #number of pages
            try:
                lista.append(soup.find('span', itemprop='numberOfPages').text.strip().split()[0])
            except:
                lista.append('')

                #Publishing Date
            try:
                a=soup.find_all('div', class_='row')[1].text
                match_obj = re.split('Published', re.split('by', a)[0])[1]
                lista.append(match_obj.strip())
            except:
                lista.append('')

                #characters
            try:
                l1=[]
                for d in soup.find_all('a',href=re.compile(r'/characters/*')):
                    l1.append(d.text)
                    s1=",".join(l1)
                lista.append(s1)
            except:
                lista.append('')

                #setting
            try:
                l2=[]
                for e in soup.find_all('a',href=re.compile(r'/places/*')):
                    l2.append(e.text)
                    s2=",".join(l2)
                lista.append(s2)
            except:
                lista.append('')

                #URL
            lista.append(soup.find('link')['href'].strip())

            path = personalPath + str(i) + '/article_' + str(i)+ '.tsv'

            with open(path, 'w', newline='',encoding="utf-8") as f_output:
                tsv_output = csv.writer(f_output, delimiter='\t')
                tsv_output.writerow(data)
                tsv_output.writerow(lista)
                f_output.close()
        
        else:
            print('This book is not in english: '+ str(i))
            
    except:
        print('Missing plot for book: '+ str(i))

In [16]:
i = 1
index = 1

new_file = open('index_books.tsv', 'w')

while i <= 30000:
    try:
        art_f = open(personalPath + str(i) + "/article_" + str(i) + ".tsv", 'r')
        art = art_f.readlines()[1]
        new_file.write(str(index) + "\t" + art)
        art_f.close()
        i += 1
        index += 1
    except:
        i += 1

new_file.close()

In [2]:
nodigit = lambda wordslist : [word for word in wordslist if word.isalpha()]


In [ ]:


f = open("books.tsv", 'r')
books = f.readlines()

new_file = open('vocabolary.tsv', 'w')

ps = PorterStemmer()

term_id = 1
document_id = 1

vocabolary = dict()
diz = defaultdict(set)

for book in books:
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+") 
    text_tokens = nodigit(tokenizer.tokenize(book.split('\t')[6]))
    tokens_without_sw = {word for word in text_tokens if not word in stopwords.words()}
    for word in tokens_without_sw:
        w = ps.stem(word.lower())
        if w not in vocabolary:
            vocabolary[w] = term_id
            diz[term_id].add(document_id)
            new_file.write(w + "\t" + str(term_id) + '\n')
            term_id += 1
        else:
            diz[vocabolary[w]].add(document_id)
    print('Finished document ' + str(document_id))
    document_id += 1
    
new_file.close()
f.close()

In [ ]:
with open("dictionary.json", "w") as outfile: 
    json.dump(dict(zip(diz.keys(), map(list, diz.values()))), outfile, indent = 4)

In [ ]:
def query():
    # input from user
    q = input().strip().split()
    # import documents 
    # --> we can optimize it loading documents out of function
    # --> but isn't very correct (formally)
    voc = pd.read_csv("vocabolary.tsv", sep='\t', header=None, error_bad_lines=False) # vocabolary
    with open('dictionary.json') as f:
        dt = json.load(f) # dictionary
    ds = pd.read_csv('index_books.tsv', header=None, sep='\t', usecols=[0,1,7,12])
    ds.rename(columns={0:'index', 1:'bookTitle', 7:'plot', 12:'url'}, inplace=True)
    # elaborate query
    # take term_id(s)
    term = list()
    for w in q:
        term.append(int(voc[voc[0]==w.lower()][1]))
    # matching documents
    doc = set(dt[str(term[0])])
    for i in range(1, len(term)):
        doc.intersection(dt[str(term[i])])
    # take row from books
    return ds[ds['index'].isin(list(doc))].head()

In [3]:
#voc = pd.read_csv("vocabolary.tsv", sep='\t', header=None, error_bad_lines=False) # vocabulary
voc = dict()
with open('vocabolary.tsv') as f:
    for col1, col2 in csv.reader(f, delimiter='\t'):
        voc[col2] = col1
        
with open('dictionary.json') as f:
    dt = json.load(f) # dictionary
    
#ds = pd.read_csv('index_books.tsv', header=None, sep='\t', usecols=[0,1,7,12])
#ds.rename(columns={0:'index', 1:'bookTitle', 7:'plot', 12:'url'}, inplace=True)

ds = dict()
with open('index_books.tsv') as f:
    for row in csv.reader(f, delimiter='\t'):
        if len(row) == 13:
            ds[row[0]] = row[7]

In [4]:
def query():
    ps = PorterStemmer()

    q = input().strip().split() # input from user
    
    ps = PorterStemmer()
    q = [ps.stem(w) for w in q]

    # elaborate query
    
    # take term_id(s)
    term = list()
    for w in q:
        try:
            term.append(int(voc[voc[0]==w.lower()][1]))
        except:
            pass
    # matching documents
    if len(term):
        doc = set(dt[str(term[0])])
        for i in range(1, len(term)):
            doc.intersection(dt[str(term[i])])
        # take row from books
        
        return ds[ds['index'].isin(list(doc))].head()
    else:
        return "There aren't documents for each word of this query"

In [5]:
def tf(doc_id, term_id):
    try:
        word = voc[term_id]
        plot = ds[str(doc_id)]

        ps = PorterStemmer()
        tokenizer = RegexpTokenizer(r"[a-zA-Z]+") 
        text_tokens = nodigit(tokenizer.tokenize(plot))
        tokens_without_sw = [ps.stem(w.lower()) for w in text_tokens if not w in stopwords.words()]

        tf = tokens_without_sw.count(word) / len(tokens_without_sw)
        return tf
    
    except:
        return -1

In [ ]:
d = dict()
num_docs = len(ds)

i = 1

for term, documents in dt.items():
    heap=[]
    for document in documents:
        idf = math.log10( num_docs / len( dt[str(term)] ) )
        t = tf(document, term)
        if t != -1:
            heapq.heappush(heap, (document, t*idf ) )
            print(t)
    
    d[key]=heap
    
    i += 1
    
    if i == 2:
        break

with open("inverted_index.json", "w") as outfile: 
    json.dump(d, outfile, indent = 4)

0.005
0.011627906976744186
0.004739336492890996
0.010416666666666666
0.004545454545454545
0.008403361344537815
0.016129032258064516
0.01098901098901099
0.010101010101010102
0.008333333333333333
0.0022471910112359553
0.011627906976744186
0.024390243902439025
0.11538461538461539
0.020833333333333332
0.017241379310344827
0.019417475728155338
0.0037735849056603774
0.01818181818181818
0.016666666666666666
0.010752688172043012
0.017241379310344827
0.01098901098901099
0.008547008547008548
0.013071895424836602
0.038461538461538464
0.0058823529411764705
0.010752688172043012
0.020833333333333332
0.010101010101010102
0.010526315789473684
0.014705882352941176
0.010526315789473684
0.011494252873563218
0.00980392156862745
0.014705882352941176
0.017857142857142856
0.007874015748031496
0.00819672131147541
0.017241379310344827
0.014285714285714285
0.01098901098901099
0.010416666666666666
0.0125
0.005555555555555556
0.006369426751592357
0.02
0.011904761904761904
0.011904761904761904
0.013888888888888888